# 🔥Causal Graph Neural Networks for Wildfire Danger Prediction🔥
Re-implementation of original work by Zhao et al.(2024) (https://arxiv.org/abs/2403.08414)

IDL S25 Group 23: Wenting Yue, Wenyu Liu, Youyou Huang (Group 23)

## Retrieve files from github repository
If `only notebook` is downloaded locally

In [1]:
import os
import sys

# Get the current working directory
print("Current working directory:", os.getcwd())
# repo = "https://github.com/youyouh511/11785_IDL_S25_Final-Project.git"
# !git clone {repo}
!git pull

Current working directory: /home/wenzheng/Wenyu/11785_IDL_S25_Final-Project


Already up to date.


# Set up

## Environment

Environment setup
```bash
conda env create -f env.yml
```

Activate environment and check device
```bash
conda activate idl
nvidia-smi
python -c "import torch; print('CUDA available:', torch.cuda.is_available())"
```

# Imports

In [2]:
from data import (
    JsonFireDataset
)
from model import (
    AdjacencyMatrix,
    TemporalLSTM,
    CausalGNN
)
# from train import (
    
# )
# from utils import (
# )


import numpy as np
import tqdm
import matplotlib.pyplot as plt
import json
import zipfile
import torch
import requests
import xarray as xr
import yaml
from torchinfo import summary
import shutil
import wandb
import time
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


/home/wenzheng/.conda/envs/hw4/lib/python3.12/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


Using device: cuda


# Config

In [3]:
%%writefile config.yaml

###### Dataset
    root                    : "./data"
    train_json_path         : "train.json"
    val_json_path           : "val.json"
    test_json_path          : "test.json"
    matrix_json_path        : "matrix.json"
    subset                  : 1.0
    batch_size              : 128
    NUM_WORKERS             : 4

    ### Target threshold
    fire_threshold          : 10


###### Model
    ### Adjacency matrix
    local_var_lag           : 8
    oci_var_lag             : 31
    max_lag                 : 312
    independence_test       : "ParCorr"
    tau_max                 : 186
    pc_alpha                : 0.05
    mask_target             : True

    ### Temporal LSTM
    lstm_layer              : 1
    hidden_dim              : 256

    ### GNN
    gnn_nodes               : 7


###### Training
    epochs                  : 30

    lr                      : 1.0e-5
    min_lr                  : 1.0e-9
    
    optimizer               : "Adam"
    betas                   : [0.9, 0.999]
    eps                     : 1.0e-8
    weight_decay            : 5.0e-6

    lr_scheduler            : "CosineAnnealingLR"
    patience                : 10
    early_stop              : True

    save_model              : True
    save_model_path         : "./checkpoints"
    load_model              : False
    load_model_path         : "./checkpoints/best.pth"
    wandb_log               : True
    wandb_project           : "IDL_Final"
    wandb_run_id            : None

Overwriting config.yaml


In [4]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

config

{'root': './data',
 'train_json_path': 'train.json',
 'val_json_path': 'val.json',
 'test_json_path': 'test.json',
 'matrix_json_path': 'matrix.json',
 'subset': 1.0,
 'batch_size': 128,
 'NUM_WORKERS': 4,
 'fire_threshold': 10,
 'local_var_lag': 8,
 'oci_var_lag': 31,
 'max_lag': 312,
 'independence_test': 'ParCorr',
 'tau_max': 186,
 'pc_alpha': 0.05,
 'mask_target': True,
 'lstm_layer': 1,
 'hidden_dim': 256,
 'gnn_nodes': 7,
 'epochs': 30,
 'lr': 1e-05,
 'min_lr': 1e-09,
 'optimizer': 'Adam',
 'betas': [0.9, 0.999],
 'eps': 1e-08,
 'weight_decay': 5e-06,
 'lr_scheduler': 'CosineAnnealingLR',
 'patience': 10,
 'early_stop': True,
 'save_model': True,
 'save_model_path': './checkpoints',
 'load_model': False,
 'load_model_path': './checkpoints/best.pth',
 'wandb_log': True,
 'wandb_project': 'IDL_Final',
 'wandb_run_id': 'None'}

# Data Retrieval & Pre-process
Refer to data_preprocessing.ipynb

# Datasets

In [5]:
local_keys  = ["T2M_MEAN","TP","VPD"]
oci_keys    = ["OCI_NAO", "OCI_NINA34_ANOM", "OCI_AO"]

train_ds = JsonFireDataset(
    json_path   = "data/train.json",
    local_keys  = local_keys,
    oci_keys    = oci_keys
)
val_ds = JsonFireDataset(
    json_path   = "data/val.json",
    local_keys  = local_keys,
    oci_keys    = oci_keys
)
test_ds = JsonFireDataset(
    json_path   = "data/test.json",
    local_keys  = local_keys,
    oci_keys    = oci_keys
)

# Model

## Adjacency Matrix

In [6]:
"""## Filter a subset of matrix samples from matrix.json
subset_frac         = 0.001
rng_seed            = 11785
input_matrix_file   = "data/matrix.json"
subset_matrix_file  = f"data/matrix_{subset_frac}_{rng_seed}.json"

AdjacencyMatrix.sample_json_file(
    subset_frac  = subset_frac,
    rng_seed     = rng_seed,
    input_path   = input_matrix_file,
    output_path  = subset_matrix_file,
)"""

'## Filter a subset of matrix samples from matrix.json\nsubset_frac         = 0.001\nrng_seed            = 11785\ninput_matrix_file   = "data/matrix.json"\nsubset_matrix_file  = f"data/matrix_{subset_frac}_{rng_seed}.json"\n\nAdjacencyMatrix.sample_json_file(\n    subset_frac  = subset_frac,\n    rng_seed     = rng_seed,\n    input_path   = input_matrix_file,\n    output_path  = subset_matrix_file,\n)'

## Causal GNN

In [7]:
from torch.utils.data import DataLoader
import pandas as pd

In [8]:
train_loader = DataLoader(
        train_ds,
        batch_size=256,
        shuffle=True,
        num_workers=4,    # adjust to your machine
        pin_memory=True,  # if you’re on GPU
    )
val_loader = DataLoader(
        val_ds,
        batch_size=256,
        shuffle=False,
        num_workers=4,    # adjust to your machine
        pin_memory=True,  # if you’re on GPU
    )
test_loader = DataLoader(
        test_ds,
        batch_size=256,
        shuffle=False,
        num_workers=4,    # adjust to your machine
        pin_memory=True,  # if you’re on GPU
    )

In [9]:
"""matrix_builder = AdjacencyMatrix(, independence_test="ParCorr",tau_max=23)
output, varlist = matrix_builder.gen_adj_matrix("val", "mean", True, "target", True)
print(varlist)"""

'matrix_builder = AdjacencyMatrix(, independence_test="ParCorr",tau_max=23)\noutput, varlist = matrix_builder.gen_adj_matrix("val", "mean", True, "target", True)\nprint(varlist)'

In [10]:
# Step 1: Load CSV into pandas DataFrame
df = pd.read_csv("data/adj_matrix_0.1_11785.csv", index_col=0)

# Step 2: Desired new order
new_order = ["T2M_MEAN", "TP", "VPD", "OCI_NAO", "OCI_NINA34_ANOM", "OCI_AO"]

# Step 3: Reorder rows and columns
df_reordered = df.loc[new_order, new_order]

# Step 4: Save (optional)
df_reordered.to_csv("data/adj_matrix_reordered_stripped.csv", index=False, header=False)

In [11]:
import torch
import numpy as np

matrix_np = np.loadtxt("data/adj_matrix_reordered_stripped.csv", delimiter=",")
# Step 2: Convert to torch tensor (float32 for neural network use)
matrix_tensor = torch.tensor(matrix_np, dtype=torch.float32)

In [12]:
model = CausalGNN(
    adj_matrix=matrix_tensor,
    num_nodes=config['gnn_nodes'],
    hidden_dim=config['hidden_dim']
).to(device)

pmodel_stats = summary(model,     # which columns to show
    col_width=20,
    verbose=2)
print(pmodel_stats)


Layer (type:depth-idx)                   Param #
CausalGNN                                --
├─TemporalLSTM: 1-1                      --
│    └─lstm.weight_ih_l0                 ├─1,024
│    └─lstm.weight_hh_l0                 ├─262,144
│    └─lstm.bias_ih_l0                   ├─1,024
│    └─lstm.bias_hh_l0                   └─1,024
│    └─LSTM: 2-1                         265,216
│    │    └─weight_ih_l0                 ├─1,024
│    │    └─weight_hh_l0                 ├─262,144
│    │    └─bias_ih_l0                   ├─1,024
│    │    └─bias_hh_l0                   └─1,024
├─DenseGCNConv: 1-2                      512
│    └─bias                              ├─512
│    └─lin.weight                        └─131,072
│    └─Linear: 2-2                       131,072
│    │    └─weight                       └─131,072
├─GraphNorm: 1-3                         1,536
│    └─weight                            ├─512
│    └─bias                              ├─512
│    └─mean_scale                 

## Trainer

In [13]:
from train import train

# 1) 先实例化一个 Trainer 对象
trainer = train.Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader
)

# 2) 调用 fit() 开始训练（30 epochs 只是举例，改成你想要的次数）
trainer.fit(num_epochs=30)

# 3) 最后调用 test() 在测试集上评估性能
trainer.test()

Training:   0%|          | 0/30 [00:00<?, ?it/s]

Ep1 | tr loss 0.411 acc 0.844 | val loss 0.327 auroc 0.892 auprc 0.612:   3%|▎         | 1/30 [00:01<00:38,  1.34s/it]


Epoch 1:
  Train → loss 0.4112, acc 0.8445
  Val   → loss 0.3273, acc 0.8580, AUROC 0.8919, AUPRC 0.6115


Ep2 | tr loss 0.273 acc 0.883 | val loss 0.289 auroc 0.910 auprc 0.614:   7%|▋         | 2/30 [00:02<00:33,  1.19s/it]


Epoch 2:
  Train → loss 0.2731, acc 0.8834
  Val   → loss 0.2890, acc 0.8653, AUROC 0.9095, AUPRC 0.6141


...

Ep29 | tr loss 0.104 acc 0.959 | val loss 0.298 auroc 0.921 auprc 0.656:  97%|█████████▋| 29/30 [00:32<00:01,  1.13s/it]


Epoch 29:
  Train → loss 0.1039, acc 0.9594
  Val   → loss 0.2977, acc 0.8765, AUROC 0.9207, AUPRC 0.6559


Ep30 | tr loss 0.099 acc 0.963 | val loss 0.316 auroc 0.914 auprc 0.659: 100%|██████████| 30/30 [00:33<00:00,  1.13s/it]



Epoch 30:
  Train → loss 0.0987, acc 0.9630
  Val   → loss 0.3159, acc 0.8796, AUROC 0.9145, AUPRC 0.6591

🟢 Best Val AUROC: 0.9404


Test → loss 0.2758, acc 0.8940, AUROC 0.9329, AUPRC 0.7307
